# TODO

1. NdBSpline: a tuple of non-zero b-spline vectors
2. Tests with 2D and 3D
3. vector of `x`   + tests
4. trailing dimensions of `c`: reshape, loop, reshape back + tests
5. get rid of `itertools.product` : iterate over c.flat (subarray!)

In [1]:
def B(x, k, i, t):
    if k == 0:
        return 1.0 if t[i] <= x < t[i+1] else 0.0
    if t[i+k] == t[i]:
        c1 = 0.0
    else:
        c1 = (x - t[i])/(t[i+k] - t[i]) * B(x, k-1, i, t)
    if t[i+k+1] == t[i+1]:
        c2 = 0.0
    else:
        c2 = (t[i+k+1] - x)/(t[i+k+1] - t[i+1]) * B(x, k-1, i+1, t)
    return c1 + c2

def bspline(x, t, c, k):
    n = len(t) - k - 1
    assert (n >= k+1) and (len(c) >= n)
    return sum(c[i] * B(x, k, i, t) for i in range(n))

In [63]:
def _naive_eval(x, t, c, k):
    """
    Naive B-spline evaluation. Useful only for testing!
    """
    if x == t[k]:
        i = k
    else:
        i = np.searchsorted(t, x) - 1
    assert t[i] <= x <= t[i+1]
    assert i >= k and i < len(t) - k
    return sum(c[i-j] * B(x, k, i-j, t) for j in range(0, k+1))

In [65]:
_naive_eval(1.5, spl.t, spl.c, 3), spl(1.5)

(3.3749999999999996, array(3.375))

In [80]:
import numpy as np
from scipy.interpolate import make_interp_spline

def prod(iterable):
    """
    Product of a sequence of numbers.

    Faster than np.prod for short lists like array shapes, and does
    not overflow if using Python integers.
    """
    product = 1
    for x in iterable:
        product *= x
    return product

In [164]:
x = np.arange(6)
y = x**3
spl = make_interp_spline(x, y, k=3)

In [7]:
spl.t

array([0., 0., 0., 0., 2., 3., 5., 5., 5., 5.])

In [10]:
spl.c

array([ 0.00000000e+00, -8.11457125e-16,  1.61091184e-15,  3.00000000e+01,
        7.50000000e+01,  1.25000000e+02])

In [165]:
y_1 = x**3 + 2*x
spl_1 = make_interp_spline(x, y_1, k=3)
spl_1.t, spl_1.c

(array([0., 0., 0., 0., 2., 3., 5., 5., 5., 5.]),
 array([  0.        ,   1.33333333,   3.33333333,  36.66666667,
         83.66666667, 135.        ]))

In [33]:
# make the outer product of 1D coefficients

c2 = np.outer(spl_1.c, spl.c)

with np.printoptions(suppress=True, linewidth=100):
    print(c2)

[[    0.            -0.             0.             0.             0.             0.        ]
 [    0.            -0.             0.            40.           100.           166.66666667]
 [    0.            -0.             0.           100.           250.           416.66666667]
 [    0.            -0.             0.          1100.          2750.          4583.33333333]
 [    0.            -0.             0.          2510.          6275.         10458.33333333]
 [    0.            -0.             0.          4050.         10125.         16875.        ]]


In [29]:
def bspline2(xy, t, c, k):
    x, y = xy
    tx, ty = t
    nx = len(tx) - k - 1
    assert (nx >= k+1)
    ny = len(ty) - k - 1
    assert (ny >= k+1)
    return sum(c[ix, iy] * B(x, k, ix, tx) * B(y, k, iy, ty) for ix in range(nx) for iy in range(ny))

In [34]:
for x0, y0 in [(1.5, 0.5), (2.5, 1), (0.5, 1.5)]:
    print((x0**3 + 2*x0)* y0**3, ' -- ',
          bspline2((x0, y0), (spl.t, spl_1.t), c2, k=3))

0.796875  --  0.7968749999999984
20.625  --  20.625
3.796875  --  3.796875000000001


In [25]:
(0.5**3 + 2*0.5) * 1.5**3

3.796875

In [38]:
tuple(_/8 for _ in c2.strides)

(6.0, 1.0)

In [40]:
c2.shape

(6, 6)

In [44]:
len(spl.t) - spl.k - 1

6

In [150]:
class NdBSpline0:
    def __init__(self, t, c, k=3):
        """Tensor product spline object.
        
        c[i1, i2, ..., id] * B(x1, i1) * B(x2, i2) * ... * B(xd, id)
        
        
        Parameters
        ----------
        c : ndarray, shape (n1, n2, ..., nd, ...)
            b-spline coefficients
        t : tuple of 1D ndarrays
            knot vectors in directions 1, 2, ... d
            ``len(t[i]) == n[i] + k + 1``
        k : int or length-d tuple of integers
            spline degrees.
        """
        ndim = len(t)
        assert ndim <= len(c.shape)
        
        try:
            len(k)
        except TypeError:
            # make k a tuple
            k = (k,)*ndim

        self.k = tuple(operator.index(ki) for ki in k)
        self.t = tuple(np.asarray(ti, dtype=float) for ti in t)
        self.c = c

    def __call__(self, x):
        ndim = len(self.t)
        # a single evaluation point: `x` is a 1D array_like, shape (ndim,)
        assert len(x) == ndim
        
        # get the indices in an ndim-dimensional vector
        i = ['none',]*ndim
        for d in range(ndim):
            td, xd = self.t[d], x[d]
            k = self.k[d]
            
            # find the index for x[d]
            if xd == td[k]:
                i[d] = k
            else:
                i[d] = np.searchsorted(td, xd) - 1
            assert td[i[d]] <= xd <= td[i[d]+1]
            assert i[d] >= k and i[d] < len(td) - k
        i = tuple(i)
        
        # iterate over the dimensions, form linear combinations of
        # products B(x_1) * B(x_2) * ... B(x_N) of (k+1)**N b-splines
        # which are non-zero at `i = (i_1, i_2, ..., i_N)`.
        result = 0
        iters = [range(i[d] - self.k[d], i[d] + 1) for d in range(ndim)]
        for idx in itertools.product(*iters):
            term = self.c[idx] * prod(B(x[d], self.k[d], idx[d], self.t[d]) for d in range(ndim))
            result += term
        return result

In [205]:
class NdBSpline:
    def __init__(self, t, c, k=3):
        """Tensor product spline object.
        
        c[i1, i2, ..., id] * B(x1, i1) * B(x2, i2) * ... * B(xd, id)
        
        
        Parameters
        ----------
        c : ndarray, shape (n1, n2, ..., nd, ...)
            b-spline coefficients
        t : tuple of 1D ndarrays
            knot vectors in directions 1, 2, ... d
            ``len(t[i]) == n[i] + k + 1``
        k : int or length-d tuple of integers
            spline degrees.
        """
        ndim = len(t)
        assert ndim <= len(c.shape)
        
        try:
            len(k)
        except TypeError:
            # make k a tuple
            k = (k,)*ndim

        self.k = tuple(operator.index(ki) for ki in k)
        self.t = tuple(np.asarray(ti, dtype=float) for ti in t)
        self.c = c

    def __call__(self, xi):
        """Evaluate the tensor product b-spline at coordinates.
        
        Parameters
        ----------
        xi : array_like, shape(..., ndim)
            The coordinates to evaluate the interpolator at.
            This can be a list or tuple of ndim-dimensional points
            or an array with the shape (num_points, ndim).
            
        Returns
        -------
        values : ndarray, shape xi.shape[:-1] + self.c.shape[:ndim]
            Interpolated values at xi
        """
        
        ndim = len(self.t)
        # a single evaluation point: `x` is a 1D array_like, shape (ndim,)
        assert len(x) == ndim
        
        # get the indices in an ndim-dimensional vector
        i = ['none',]*ndim
        b = np.empty((ndim, max(self.k)+1), dtype=float) * np.nan
        for d in range(ndim):
            td, xd = self.t[d], x[d]
            k = self.k[d]
            
            # find the index for x[d]
            if xd == td[k]:
                i[d] = k
            else:
                i[d] = np.searchsorted(td, xd) - 1
            assert td[i[d]] <= xd <= td[i[d]+1]
            assert i[d] >= k and i[d] < len(td) - k
            
            # (k+1) b-splines which are non-zero at x[d] 
            b[d, :k+1] = [B(xd, k, j, td) for j in range(i[d]-k, i[d]+1)]   
        i = tuple(i)
        
        # iterate over the dimensions, form linear combinations of
        # products B(x_1) * B(x_2) * ... B(x_N) of (k+1)**N b-splines
        # which are non-zero at `i = (i_1, i_2, ..., i_N)`.
        result = 0
        iters = [range(i[d] - self.k[d], i[d] + 1) for d in range(ndim)]
        for idx in itertools.product(*iters):
            term = self.c[idx] * prod(b[d, idx[d] - i[d] + k] for d in range(ndim))
            result += term
        return result

In [206]:
bspl2 = NdBSpline((spl.t, spl_1.t), c2, k=3)

for x0, y0 in [(1.5, 2.5), (2.5, 1), (0.5, 1.5)]:
    print((x0**3 + 2*x0)* y0**3, ' -- ',
          bspline2((x0, y0), (spl.t, spl_1.t), c2, k=3), '--',
         ## naive_B_2((x0, y0), (spl.t, spl_1.t), c2, k=3), )
          bspl2((x0, y0))
         )

99.609375  --  99.60937499999999 -- 99.60937499999997
20.625  --  20.625 -- 20.625
3.796875  --  3.796875000000001 -- 3.796875000000001


In [207]:
spl2 = NdBSpline((spl.t, spl_1.t), c2, k=3)

spl2((1.5, 2.5))

99.60937499999997

In [108]:
spl.t, spl_1.t

(array([0., 0., 0., 0., 2., 3., 5., 5., 5., 5.]),
 array([0., 0., 0., 0., 2., 3., 5., 5., 5., 5.]))

In [145]:
def naive_B_2(xy, t, c, k):
    """
    Naive B-spline evaluation. Useful only for testing!
    """
    x, y = xy
    tx, ty = t
    
    if x == tx[k]:
        ix = k
    else:
        ix = np.searchsorted(tx, x) - 1
    assert tx[ix] <= x <= tx[ix + 1]
    assert ix >= k and ix < len(tx) - k

    if y == ty[k]:
        iy = k
    else:
        iy = np.searchsorted(ty, y) - 1
    assert ty[iy] <= y <= ty[iy + 1]
    assert iy >= k and iy < len(ty) - k
    
    print(ix, iy)
    
    res = sum(c[ix-jx, iy - jy] * B(x, k, ix-jx, tx) * B(y, k, iy-jy, ty)
               for jx in range(0, k+1) for jy in range(0, k+1))
    
    result = 0
    for jx in range(ix-k, ix+1):
        for jy in range(iy-k, iy+1):
            print(jx, ix - jx)
            result += c[jx, jy] * B(x, k, jx, tx) * B(y, k, jy, ty)
            
    assert np.allclose(result,  res) #, result-res
    return result

In [146]:
x0, y0 = (1.5, 2.5)
naive_B_2((x0, y0), (spl.t, spl_1.t), c2, k=3),

3 4
0 3
0 3
0 3
0 3
1 2
1 2
1 2
1 2
2 1
2 1
2 1
2 1
3 0
3 0
3 0
3 0


(99.60937499999999,)

In [208]:
np.array([(1.5, 2.5), (2.5, 1), (0.5, 1.5)]).shape

(3, 2)

In [210]:
np.array((1.5, 2.5)).shape

(2,)

In [79]:
k = 3
p = itertools.product(*(range(0, k+1),)*3)
for j, tpl in enumerate(p):
    print(j, tpl, a[tpl])

0 (0, 0, 0) 0
1 (0, 0, 1) 1
2 (0, 0, 2) 2
3 (0, 0, 3) 3
4 (0, 1, 0) 4
5 (0, 1, 1) 5
6 (0, 1, 2) 6
7 (0, 1, 3) 7
8 (0, 2, 0) 8
9 (0, 2, 1) 9
10 (0, 2, 2) 10
11 (0, 2, 3) 11
12 (0, 3, 0) 12
13 (0, 3, 1) 13
14 (0, 3, 2) 14
15 (0, 3, 3) 15
16 (1, 0, 0) 16
17 (1, 0, 1) 17
18 (1, 0, 2) 18
19 (1, 0, 3) 19
20 (1, 1, 0) 20
21 (1, 1, 1) 21
22 (1, 1, 2) 22
23 (1, 1, 3) 23
24 (1, 2, 0) 24
25 (1, 2, 1) 25
26 (1, 2, 2) 26
27 (1, 2, 3) 27
28 (1, 3, 0) 28
29 (1, 3, 1) 29
30 (1, 3, 2) 30
31 (1, 3, 3) 31
32 (2, 0, 0) 32
33 (2, 0, 1) 33
34 (2, 0, 2) 34
35 (2, 0, 3) 35
36 (2, 1, 0) 36
37 (2, 1, 1) 37
38 (2, 1, 2) 38
39 (2, 1, 3) 39
40 (2, 2, 0) 40
41 (2, 2, 1) 41
42 (2, 2, 2) 42
43 (2, 2, 3) 43
44 (2, 3, 0) 44
45 (2, 3, 1) 45
46 (2, 3, 2) 46
47 (2, 3, 3) 47
48 (3, 0, 0) 48
49 (3, 0, 1) 49
50 (3, 0, 2) 50
51 (3, 0, 3) 51
52 (3, 1, 0) 52
53 (3, 1, 1) 53
54 (3, 1, 2) 54
55 (3, 1, 3) 55
56 (3, 2, 0) 56
57 (3, 2, 1) 57
58 (3, 2, 2) 58
59 (3, 2, 3) 59
60 (3, 3, 0) 60
61 (3, 3, 1) 61
62 (3, 3, 2) 62
63 (3, 3, 3)